In [ ]:
import pyomo.environ as pyo

model = pyo.AbstractModel()

# Parámetros
model.p1 = pyo.Param()
model.p2 = pyo.Param()
model.p3 = pyo.Param()
model.p4 = pyo.Param()
model.p5 = pyo.Param()
model.p6 = pyo.Param()
model.p7 = pyo.Param()
model.p8 = pyo.Param()
model.p9 = pyo.Param()
model.ps = pyo.Param()

# Definir Block para la población de ciervos
model.population = pyo.Block()

# Variables dentro del bloque de población
model.population.f = pyo.Var(initialize=20, within=pyo.PositiveReals)
model.population.d = pyo.Var(initialize=20, within=pyo.PositiveReals)
model.population.b = pyo.Var(initialize=20, within=pyo.PositiveReals)
model.population.hf = pyo.Var(initialize=20, within=pyo.PositiveReals)
model.population.hd = pyo.Var(initialize=20, within=pyo.PositiveReals)
model.population.hb = pyo.Var(initialize=20, within=pyo.PositiveReals)
model.population.br = pyo.Var(initialize=1.5, within=pyo.PositiveReals)

# Definir Block para recursos y consumo de alimentos
model.resources = pyo.Block()

# Variables dentro del bloque de recursos
model.resources.c = pyo.Var(initialize=500000, within=pyo.PositiveReals)

# Función objetivo
def obj_rule(m):
    return 10 * m.population.hb + m.population.hd + m.population.hf
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Restricciones dentro del bloque de población
def f_bal_rule(m):
    return m.population.f == m.p1 * m.population.br * (m.p2/10.0 * m.population.f + m.p3 * m.population.d) - m.population.hf
model.population.f_bal = pyo.Constraint(rule=f_bal_rule)

def d_bal_rule(m):
    return m.population.d == m.p4 * m.population.d + m.p5/2.0 * m.population.f - m.population.hd
model.population.d_bal = pyo.Constraint(rule=d_bal_rule)

def b_bal_rule(m):
    return m.population.b == m.p6 * m.population.b + m.p5/2.0 * m.population.f - m.population.hb
model.population.b_bal = pyo.Constraint(rule=b_bal_rule)

# Restricciones dentro del bloque de recursos
def food_cons_rule(m):
    return m.resources.c == m.p7 * m.population.b + m.p8 * m.population.d + m.p9 * m.population.f
model.resources.food_cons = pyo.Constraint(rule=food_cons_rule)

def supply_rule(m):
    return m.resources.c <= m.ps
model.resources.supply = pyo.Constraint(rule=supply_rule)

# Restricción de la tasa de nacimientos
def birth_rule(m):
    return m.population.br == 1.1 + 0.8 * (m.ps - m.resources.c) / m.ps
model.population.birth = pyo.Constraint(rule=birth_rule)

# Restricción de la proporción mínima de machos
def minbuck_rule(m):
    return m.population.b >= 1.0/5.0 * (0.4 * m.population.f + m.population.d)
model.population.minbuck = pyo.Constraint(rule=minbuck_rule)

# Crear la instancia del modelo y resolver
instance = model.create_instance("DeerProblem.dat")
status = pyo.SolverFactory("ipopt").solve(instance)
pyo.assert_optimal_termination(status)
instance.pprint()
